![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 10 - Selección de hiperparámetros

Cuando se está entrenando un modelo para resolver una determinada tarea hay dos elementos que cobran especial importancia:
1. Los **parámetros**. Son los valores que durante el proceso de entrenamiento del modelo deben aprenderse. Los parámetros serán los responsables de que el sistema tenga (o no) un buen rendimiento en las predicciones. Por ejemplo, cuando se hace regresión lineal, los coeficientes que se aprenden son lo que aquí llamamos parámetros; también lo serían los pesos de una red neuronal, los valores de corte de un árbol de decisión,...
2. Los **hiperparámetros**. Son características particulares de cada algoritmo que pueden hacer que el comportamiento del modelo entrenado sea diferente. Por ejemplo, en el KNN el número de vecinos (`n_neighbors`) es un hiperparámetro y como vimos en la práctica anterior, el rendimiento del modelo será diferente.

Una regla sencilla para saber qué es un hiperparámetro y qué es un parámetro es la siguiente: **las decisiones que tomamos antes de entrenar un modelo son o afectan a hiperparámetros, mientras que los términos que aprende el modelo durante el entrenamiento son parámetros.**

El KNN tiene varios hiperparámetros:
- el número de vecinos (`n_neighbors`). Debe ser un número entero mayor o igual que 1
- si se pondera el voto de los vecinos (`weights`). Puede tomar los valores `uniform` o `distance`, donde el primero indica que todos los votos pesan lo mismo y el segundo que se pondera en función de la distancia.
- la distancia utilizada (`p`). Por defecto se utiliza la distancia de Minkowski, que cuando `p=1` se corresponde con la distancia Manhattan y cuando `p=2` con la distancia Euclídea. `p` debe ser un valor  entero mayor o igual que 1.

$$D(a,b) = \left(\sum_{i}(a_i-b_i)^p\right)^{1/p}$$

Sin embargo el KNN no tiene parámetros, ya que no tiene nada que aprender puesto que simplemente memoriza los ejemplos para calcular distancias durante la predicción.

En esta práctica vamos a ver cómo se seleccionan los hiperparámetros de los modelos. 

## 10.1 Grid Search

Cuando queremos utilizar una búsqueda de hiperparámetros por fuerza bruta donde queremos que se prueben todas las combinaciones posibles para los valores de hiperparámetros que indicamos, entonces debemos utilizar una *grid search*, que en `sklearn` se implementa mediante la clase `GridSearchCV`.

La `GridSearchCV` realizará una validación cruzada para cada una de las combinaciones de hiperparámetros posibles y se quedará con aquella combinación que produzca el mejor rendimiento.

Vamos a ver cómo realizar esta búsqueda de hiperparámetros utilizando nuevamente el conjunto `ionosphere`, para ello lo primero que vamos a hacer es cargar el conjunto:

In [ ]:
# se importan las librerías
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint

# se llama a la función read_csv
# no tiene missing y las columnas están separadas por ','
# tampoco cabecera, así que hay que dar nombre a las columnas (como en el names no vienen indicados creamos nombres)
cabecera = ['atr'+str(x) for x in range(1,35)]
cabecera.append('clase')
df = pd.read_csv('ionosphere.data', names=cabecera)
filas, columnas = df.shape

# la clase está en la última columna 
# separamos los atributos y los almacenamos en X
X = df.iloc[:,0:(columnas-1)]
display(X)

class_enc = preprocessing.LabelEncoder()
df['clase'] = class_enc.fit_transform(df['clase'])

# separamos la clase y la almacenamos en Y
y = df.iloc[:,(columnas-1)]
display(y)

Una vez cargado el conjunto de datos vamos a crear un sistema KNN y a configurar los hiperparámetros que queremos probar:

In [ ]:
# creamos una instancia del KNN
knn_sis = KNeighborsClassifier()

# se definen los valores de los hiperparámetros que se quieren  probar
weights = ['uniform', 'distance'] # weights : {'uniform', 'distance'}
p = [1, 2, 3] # p : int, default=2 => Euclídea
n_neighbors = [1, 2, 3, 4, 5]

# y se introducen en un diccionario
hyperparameters = dict(weights=weights, p=p, n_neighbors=n_neighbors)

display(hyperparameters)

La manera más sencilla de hacerlo es como vemos en el código: para cada hiperparámetro se indican los valores que se quieren probar en una lista y luego se crea un diccionario con todos los hiperparámetros.

Posteriormente, debemos crear la grid search:

In [ ]:
# se crea un generador de folds estratificados partiendo el conjunto en 5 trozos
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# creamos una grid search para el KNN donde le pasamos los hiperparámetros que queremos probar
gs = GridSearchCV(knn_sis, hyperparameters, scoring='accuracy', cv=folds, verbose=3, n_jobs=-1)

Se debe indicar cuál es el algoritmo y cuáles son los parámetros a probar. También podemos indicarle qué métrica queremos utilizar para medir el rendimiento y los folds que vamos a utilizar en la validación cruzada (es importante garantizar el barajado de alguna manera para evitar sesgos). 

`verbose=3` servirá para ir viendo una traza de lo que `GridSearchCV()` va haciendo; podemos utilizar otros niveles inferiores de `verbose` (2, 1 o 0) si no queremos tanta información.

En las búsquedas de hiperparámetros es recomendable paralelizar las ejecuciones, por eso se suele utilizar `n_jobs=-1`. En el ejemplo que estamos llevando a cabo, para el hiperparámetro `weights` queremos probar 2 valores, para `p` 3 valores y para `n_neighbors` 5 valores. Esto hace un total de $2 \times 3 \times 5 = 30$ posibles combinaciones. Como cada combinación se prueba con una validación cruzada de 5 folds, se realizarán en total $30 \times 5 = 150$ entrenamientos diferentes.

Para realizar la búsqueda de los hiperparámetros debemos entrenar:

In [ ]:
# ejecutamos la búsqueda
res_gs = gs.fit(X, y)

Aparecerá una línea por cada uno de los entrenamientos donde se detallará a qué combinación corresponde. Si no se hubiese paralelizado el entrenamiento aparecerían las ejecuciones ordenadas.

Una vez finalizada la búsqueda, `GridSearchCV()` realiza un entrenamiento utilizando todos los ejemplos disponibles y con la combinación de hiperparámetros que mejor rendimiento presentó en validación cruzada. 

En https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html podemos ver que el resultado de la `GridSearchCV()` contiene mucha información. Podemos acceder a tiempos de ejecución o a resultados de cada validación cruzada:

In [ ]:
# combinaciones probadas
print("Combinaciones de hiperparámetros probadas:\n", res_gs.cv_results_['params'])

# resultados
print("Accuracy media de cada combinación:\n", res_gs.cv_results_['mean_test_score'])
print("Desviación en cada combinación:\n", res_gs.cv_results_['std_test_score'])

Podemos también consultar cuál ha sido el mejor rendimiento obtenido y la combinación de hiperparámetros que lo propició. Además, `best_estimator_` contiene el modelo entrenado con la mejor combinación y utilizando todos los ejemplos:

In [ ]:
print("Mejor combinación de hiperparámetros:", res_gs.best_params_)
print("Mejor rendimiento obtenido: %.4f" % res_gs.best_score_)

# modelo entrenado
best_model = res_gs.best_estimator_
print(best_model)

Al imprimir `best_model` no se especifica `weights='uniform'` porque ese es su valor por defecto en `KNeighborsClassifier()`.

## 10.2 Búsqueda aleatoria

Cuando el espacio de búsqueda es demasiado grande y no se pueden probar todas las combinaciones posibles por resultar infinitas o porque nos extenderíamos demasiado en el tiempo, entonces debemos optar por realizar una búsqueda en la que se prueben combinaciones seleccionadas al azar.

Sobre el mismo ejemplo que estamos siguiendo, imaginemos que queremos probar combinaciones de hiperparámetros utilizando valores para el número de vecinos entre 1 y 50, para la `p` de Minkowski valores entre 1 y 10 y ponderando o no las distancias. Esto haría un total de $50 \times 10 \times 2 = 1000$ posibles combinaciones de hiperparámetros. 

Si probar estas 1000 combinaciones es muy costoso, tenemos la posibilidad de utilizar una `RandomizedSearchCV()` que probará un número determinado de combinaciones de hiperparámetros: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV 

A la hora de definir los valores posibles para los hiperparámetros, tendremos la posibilidad de pasarle una distribución para la generación de valores para los hiperparámetros:

In [ ]:
# distribución uniforme entre 1 y 50
dist_n_neighbors = randint(1, 50)

# distribución uniforme entre 1 y 10
dist_p = randint(1,10)

# se especifican los valores posibles y muestreará de forma uniforme
weights = ['uniform', 'distance'] 

# se crea el diccionario de hiperparámetros
hyperparameters = dict(weights=weights, p=dist_p, n_neighbors=dist_n_neighbors)

Para generar las distribuciones podemos acudir a la librería `scipy` donde encontramos funciones como `randint()` que genera valores enteros en un intervalo, `uniform()` que lo hace para números reales o `norm()` que los genera siguiendo una distribución normal.

Una vez que tenemos el diccionario de hiperparámetros preparado tenemos que crear el `RandomizedSearchCV()`:

In [ ]:
rs = RandomizedSearchCV(knn_sis, hyperparameters, scoring='accuracy', random_state=1234, n_iter=100, cv=folds, verbose=3, n_jobs=-1)

Además de los datos que ya se pasaban a `GridSearchCV()` en este caso debemos pasarle `random_state` si queremos que los resultados sean reproducibles y el parámetro `n_iter` con el que le indicamos el número de combinaciones que queremos probar. En este ejemplo se indica `n_iter=100` con lo que se probaría un 10% de total de combinaciones posibles. `RandomizedSearchCV()` efectúa un muestreo sin reemplazamiento.

Nos queda entrenar:

In [ ]:
# entrenamos
res_rs = rs.fit(X, y)

Y ver los resultados:

In [ ]:

print("Hiperparámetros:", res_rs.cv_results_['params'])
print("Accuracy:", res_rs.cv_results_['mean_test_score'])
print("Desviación:", res_rs.cv_results_['std_test_score'])
print("Mejor combinación de hiperparámetros:", res_rs.best_params_)
print("Mejor rendimiento obtenido: %.4f" % res_rs.best_score_)

Vemos que obtenemos un resultado un poco peor que con `GridSearchCV()`, sin embargo, hemos explorado un espacio mucho más grande.

A veces, se utiliza `RandomizedGridSearch()` para acotar el espacio de búsqueda y posteriormente se realiza una `GridSearchCV()` para realizar una búsqueda más exhaustiva en la zona del espacio de hiperparámetros que indique la `RandomizedGridSearch()`.

## Ejercicios

1. Carga el fichero **heart_failure_clinical_records_dataset.csv** (es un archivo de texto). 
2. Realiza una búsqueda aleatoria utilizando valores para el número de vecinos entre 1 y 50, para la `p` de Minkowski valores entre 1 y 10 y ponderando o no las distancias. **OJO**, ten en cuenta que los atributos tienen escalas diferentes, así que deberás crear un pipeline.
3. Una vez acotado el espacio de búsqueda, realiza una búsqueda más exhaustiva utilizando una `GridSearchCV()`.


Estos ejercicios no es necesario entregarlos.